In [4]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds


In [5]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.ROZIAG_3.0.1/mnist-train.tfrecord*...:   0%|          | 0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.ROZIAG_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [6]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [8]:
num_validation_samples=0.1*mnist_info.splits['train'].num_examples
num_validation_samples=tf.cast(num_validation_samples, tf.int64)

In [9]:
num_test_samples=mnist_info.splits['test'].num_examples
num_test_samples=tf.cast(num_test_samples, tf.int64)

In [10]:
def scale(image, label):
    image=tf.cast(image, tf.float32)
    image/=255.
    return image, label

In [11]:
scaled_trained_and_validation_data=mnist_train.map(scale)

In [13]:
test_data=mnist_test.map(scale)

In [14]:
Buffersize=10000
shuffled_train_and_validation_data=scaled_trained_and_validation_data.shuffle(Buffersize)

In [15]:
validation_data=shuffled_train_and_validation_data.take(num_validation_samples)

In [16]:
train_data=shuffled_train_and_validation_data.skip(num_validation_samples)

In [17]:
Batchsize=100
train_data=train_data.batch(Batchsize)
validation_data=validation_data.batch(num_validation_samples)
test_data=test_data.batch(num_test_samples)

In [18]:
validattion_inputs, validation_targets=next(iter(validation_data))

In [23]:
input_size=784
output_size=10
hidden_layer_size=100

model=tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [24]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [25]:
NUM_EPOCHS=5
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validattion_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 5s - 10ms/step - accuracy: 0.9035 - loss: 0.3419 - val_accuracy: 0.9513 - val_loss: 0.1788
Epoch 2/5
540/540 - 6s - 12ms/step - accuracy: 0.9594 - loss: 0.1380 - val_accuracy: 0.9660 - val_loss: 0.1200
Epoch 3/5
540/540 - 4s - 8ms/step - accuracy: 0.9712 - loss: 0.0947 - val_accuracy: 0.9683 - val_loss: 0.1038
Epoch 4/5
540/540 - 4s - 8ms/step - accuracy: 0.9782 - loss: 0.0724 - val_accuracy: 0.9775 - val_loss: 0.0765
Epoch 5/5
540/540 - 6s - 10ms/step - accuracy: 0.9826 - loss: 0.0585 - val_accuracy: 0.9805 - val_loss: 0.0656


In [26]:
test_loss, test_accuracy=model.evaluate(test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 857ms/step - accuracy: 0.9720 - loss: 0.0919


In [28]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.09. Test accuracy: 97.20%
